
## Nearest Neighbor Search

Nearest Neighbor Search is the process of finding **the vectors most similar** to a query vector inside a large collection of embeddings.

In Generative AI, embeddings represent the *meaning* of:

* text
* sentences
* documents
* images
* audio

NNS retrieves the items whose embeddings lie **closest in vector space** to the query.

---

### Why NNS Is Critical in Generative AI

#### LLMs cannot “remember” or “search” billions of documents.

So we store document embeddings in a **vector database** and use NNS to retrieve the most relevant chunks.

RAG pipeline:

```
User query → embed → NNS → retrieve relevant docs → feed to LLM
```

Quality of NNS directly affects:

* factual correctness
* hallucination reduction
* search accuracy
* chatbot reliability

---

### How Nearest Neighbor Search Works

Given:

* a **query embedding** q
* a list of **indexed embeddings** D = {d₁, d₂, … dₙ}

NNS finds the top-k closest embeddings based on similarity.

Most common metric:

* **Cosine similarity (1 - angular distance)**
  Others:
* Euclidean distance
* Dot-product similarity

Example:

```
Query: "What is GDPR?"
Top neighbors:
1. Chunk about EU data protection laws
2. Chunk about privacy regulations
3. Chunk about compliance rules
```

This is **semantic search**, not keyword search.

---

### Exact Search vs Approximate Search

#### **Exact Nearest Neighbor Search (k-NN)**

* Brute force comparison: compare query with all vectors
* O(N × D) time
* Too slow for millions of vectors

Works only for small datasets.

---

#### **Approximate Nearest Neighbor Search (ANN)**

Used in all production RAG systems.

Goal:

> Return neighbors that are *almost* the best, but **much faster**.

Benefits:

* 10× to 1000× faster
* Works for millions/billions of embeddings
* Small accuracy tradeoff (<1% loss)

ANN frameworks:

* **FAISS (Meta)**
* **HNSW** (Hierarchical Navigable Small Worlds)
* **ScaNN (Google)**
* **Pinecone / Qdrant / Weaviate** (vector DBs that wrap ANN algorithms)

---

### Why ANN Is Needed (The Curse of Dimensionality)

Embeddings from LLMs are high-dimensional:

* 384
* 768
* 1536
* 4096

High-dimensional spaces break traditional search algorithms.

ANN algorithms are specifically designed to operate efficiently in **high-dimensional** embedding spaces.

---

### How ANN Algorithms Work (Intuition)

#### 🔹 **1. FAISS (IVF / PQ / Flat)**

* Clusters embeddings
* Assigns vectors to closest centroids
* Searches only relevant clusters
* Optional compression reduces RAM

#### 🔹 **2. HNSW (Graph Search)**

* Builds a graph where each vector points to its neighbors
* Query = navigate through the graph
* Extremely fast and accurate
* Used by Qdrant, Weaviate

#### 🔹 **3. ScaNN**

* Google's optimized ANN system
* Focus on TPU/CPU performance
* Used internally by YouTube, Search

---

### Nearest Neighbor Search in RAG

Here’s how NNS fits into Retrieval-Augmented Generation:

#### Step 1: Embed query

```
q = embed("How does self-attention work?")
```

#### Step 2: Query vector DB using ANN

```
results = vector_db.search(q, top_k=5)
```

#### Step 3: Retrieve best matching chunks

Results returned based on **cosine similarity**, not keyword match.

#### Step 4: Feed retrieved chunks to the LLM

```
context = concatenate(results)
answer = llm(context + user_query)
```

Without good nearest-neighbor search:

* wrong documents get retrieved
* hallucination increases
* answers become irrelevant

---

### Example (Simple Python Demonstration)

Using **FAISS**:

```python
import numpy as np
import faiss

# Example document vectors
docs = np.random.rand(1000, 128).astype("float32")

# Build FAISS index
index = faiss.IndexFlatL2(128)
index.add(docs)

# Query vector
query = np.random.rand(1, 128).astype("float32")

# Find nearest neighbors
scores, ids = index.search(query, k=3)

print("Nearest neighbors:", ids)
print("Distances:", scores)
```

---

**Summary**

| Concept                  | Explanation                                  |
| ------------------------ | -------------------------------------------- |
| **NNS**                  | Find the closest vectors to a query          |
| **Why important**        | Powers RAG, semantic search, VLM retrieval   |
| **Metric**               | Mostly cosine similarity                     |
| **Exact NNS**            | Slow, only for tiny datasets                 |
| **ANN**                  | Fast, scalable, near-perfect accuracy        |
| **FAISS / HNSW / ScaNN** | Industry-standard ANN algorithms             |
| **In RAG**               | Determines which chunks are shown to the LLM |